In [1]:
# STANDARD
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# SKYLEARN
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

# DUMMY ENCODING
import warnings
warnings.simplefilter('ignore')

In [2]:
# Read CSV, zero all NaN
data = pd.read_csv('../data/all_data.csv')
data = data.fillna(0)
data.head()


,Date,CS_Mnth,CS_Qtr,CS_Ann,CD_Mnth,CD_Qtr,CD_Ann,Enrgy_Mnth,Enrgy_Qtr,Enrgy_Ann,...,PPI_Commodity,Imp_Exp _Indx,haircut_price,cardboard_pice,Superbowl,Cosmetics_index,bodies,unidentified_bodies,underwear,beer_prct_change
0,12/1/1998,0.0000,0.0000,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,91.0,102.7,166.5,AFC,160.8,2.0,17,0.0,0.8
1,1/1/1999,0.0580,0.0000,0.0,0.0333,0.0000,0.0,-0.0013,0.0000,0.0,...,0.0,91.3,103.0,165.0,AFC,162.1,3.0,24,0.0,-1.1
2,2/1/1999,-0.0143,0.0000,0.0,0.0550,0.0000,0.0,-0.0631,0.0000,0.0,...,0.0,91.2,103.3,164.5,AFC,162.1,1.0,17,0.0,0.6
3,3/1/1999,-0.0208,0.0211,0.0,-0.0136,0.0753,0.0,-0.0064,-0.0704,0.0,...,0.0,91.5,103.7,164.6,AFC,163.6,1.0,16,0.0,-0.6
4,4/1/1999,0.0112,-0.0240,0.0,0.0563,0.0992,0.0,0.1363,0.0577,0.0,...,0.0,92.4,103.9,171.3,AFC,163.0,4.0,19,0.0,3.4


In [3]:
# NFL column to numeric data
nfl = []

for x in data["Superbowl"]:
    if x == 'AFC':
        fb = 0
        nfl.append(fb)
    elif x == 'AFL':
        fb = 1
        nfl.append(fb)
    elif x == 'NFC':
        fb = 2
        nfl.append(fb)
    
data["Superbowl"] = nfl

In [4]:
data.head()

,Date,CS_Mnth,CS_Qtr,CS_Ann,CD_Mnth,CD_Qtr,CD_Ann,Enrgy_Mnth,Enrgy_Qtr,Enrgy_Ann,...,PPI_Commodity,Imp_Exp _Indx,haircut_price,cardboard_pice,Superbowl,Cosmetics_index,bodies,unidentified_bodies,underwear,beer_prct_change
0,12/1/1998,0.0000,0.0000,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,91.0,102.7,166.5,0,160.8,2.0,17,0.0,0.8
1,1/1/1999,0.0580,0.0000,0.0,0.0333,0.0000,0.0,-0.0013,0.0000,0.0,...,0.0,91.3,103.0,165.0,0,162.1,3.0,24,0.0,-1.1
2,2/1/1999,-0.0143,0.0000,0.0,0.0550,0.0000,0.0,-0.0631,0.0000,0.0,...,0.0,91.2,103.3,164.5,0,162.1,1.0,17,0.0,0.6
3,3/1/1999,-0.0208,0.0211,0.0,-0.0136,0.0753,0.0,-0.0064,-0.0704,0.0,...,0.0,91.5,103.7,164.6,0,163.6,1.0,16,0.0,-0.6
4,4/1/1999,0.0112,-0.0240,0.0,0.0563,0.0992,0.0,0.1363,0.0577,0.0,...,0.0,92.4,103.9,171.3,0,163.0,4.0,19,0.0,3.4


In [5]:
plots_data=data.drop(['Date'], axis=1)

In [6]:
input_cols=['CS_Qtr', 'CS_Ann', 'CD_Mnth', 'CD_Qtr', 'CD_Ann',
       'Enrgy_Mnth', 'Enrgy_Qtr', 'Enrgy_Ann', 'Fncl_Mnth', 'Fncl_Qtr',
       'Fncl_Ann', 'HlthC_Mnth', 'HlthC_Qtr', 'HlthC_Ann', 'Indstrl_Mnth',
       'Indstrl_Qtr', 'Indstrl_Ann', 'Mtrls_Mnth', 'Mtrls_Qtr', 'Mtrls_Ann',
       'Tech_Mnth', 'Tech_Qtr', 'Tech_Ann', 'Util_Mnth', 'Util_Qtr',
       'Util_Ann', 'Unempl', 'Jobs_Non-Farm (000s)', 'Hrly_Wages', 'CPI_urban',
       'PPI_Commodity', 'Imp_Exp _Indx', 'haircut_price', 'cardboard_pice',
       'Superbowl', 'Cosmetics_index', 'bodies', 'unidentified_bodies',
       'underwear', 'beer_prct_change']
input_df=plots_data[input_cols]
target=plots_data['CS_Mnth']
predicted=[]
from sklearn.tree import DecisionTreeRegressor
initial_train=50
num_runs=len(input_df)-initial_train
for each_cycle in range(num_runs):
    dtr=DecisionTreeRegressor()
    dtr.fit(input_df.iloc[:initial_train+each_cycle, :], target[:initial_train+each_cycle])
    prediction=dtr.predict([input_df.iloc[initial_train+each_cycle, :]])
    print(f'Predict: {prediction} when actual is {target[initial_train+each_cycle]}')
    predicted.append(prediction)

Predict: [-0.0045] when actual is -0.0335
Predict: [0.0292] when actual is -0.0295
Predict: [-0.0335] when actual is -0.0224
Predict: [0.0405] when actual is 0.0392
Predict: [-0.0379] when actual is 0.0593
Predict: [0.0593] when actual is -0.0094
Predict: [0.0346] when actual is -0.0025
Predict: [0.01135] when actual is 0.024
Predict: [-0.0651] when actual is -0.0049
Predict: [0.0206] when actual is 0.0492
Predict: [0.0292] when actual is 0.0033
Predict: [0.0593] when actual is 0.0159
Predict: [0.0159] when actual is 0.0014
Predict: [0.0011] when actual is 0.0615
Predict: [-0.0131] when actual is -0.0125
Predict: [0.0008] when actual is 0.0206
Predict: [-0.0028] when actual is -0.0107
Predict: [0.024] when actual is 0.0039
Predict: [0.0206] when actual is -0.0553
Predict: [0.0014] when actual is 0.021
Predict: [0.0161] when actual is -0.0296
Predict: [0.0492] when actual is 0.0069
Predict: [0.0159] when actual is 0.0248
Predict: [0.021] when actual is 0.0425
Predict: [0.0069] when actu

Predict: [0.01505] when actual is 0.0677
Predict: [-0.0118] when actual is 0.0453
Predict: [0.0533] when actual is -0.0168


In [7]:
predictions = []
for i in range (len(predicted)):
    x = predicted[i][0]
    predictions.append(x)

In [8]:
Date = (data['Date'][initial_train:])
CS_Mnth = (data['CS_Mnth'][initial_train:])
len(data)

263

In [9]:
CS = []
for i in range(initial_train, len(data)):
    x = CS_Mnth[i]
    CS.append(x)

In [10]:
chart = pd.DataFrame({
    'CS_Mnth_act': CS,
    'CS_Mnth_pred': predictions,
    }, index = Date)

In [24]:
# data export function

def exportData(sector):
    plots_data=data.drop(['Date'], axis=1)
    cols = data.drop(['Date', sector], axis=1).columns
    inputs = []
    for i in range(len(cols)):
        inputs.append(cols[i])

    input_df=plots_data[inputs]
    target=plots_data[sector]
    predicted=[]
    from sklearn.tree import DecisionTreeRegressor
    initial_train=50
    num_runs=len(input_df)-initial_train
    for each_cycle in range(num_runs):
        dtr=DecisionTreeRegressor()
        dtr.fit(input_df.iloc[:initial_train+each_cycle, :], target[:initial_train+each_cycle])
        prediction=dtr.predict([input_df.iloc[initial_train+each_cycle, :]])
        predicted.append(prediction)
    
    predictions = []
    for i in range (len(predicted)):
        x = predicted[i][0]
        predictions.append(x)
        
    Date = (data['Date'][initial_train:])
    Sector = (data[sector][initial_train:])
    
    Actual = []
    for i in range(initial_train, len(data)):
        x = Sector[i]
        Actual.append(x)
        
    Sector_Code = []
    for i in range(initial_train, len(data)):
        x = sector
        Sector_Code.append(x)
        
    predictive_analytics = pd.DataFrame({
            'Date': Date,
            'Actual': Actual,
            'Predicted': predictions,
            'Sector Code': Sector_Code            
        })

    predictive_analytics=predictive_analytics.reset_index()
    predictive_analytics=predictive_analytics.drop(columns=['index'])
    
    predictive_analytics.to_csv(f"data/{sector}_predictive_analytics_data.csv")

In [12]:
# CS_Mnth
# CD_Mnth
# Enrgy_Mnth
# Fncl_Mnth
# HlthC_Mnth
# Indstrl_Mnth
# Mtrls_Mnth
# Tech_Mnth
# Util_Mnth

In [25]:
exportData('CS_Mnth')

In [26]:
exportData('CD_Mnth')

In [27]:
exportData('Enrgy_Mnth')

In [28]:
exportData('Fncl_Mnth')

In [29]:
exportData('HlthC_Mnth')

In [30]:
exportData('Indstrl_Mnth')

In [31]:
exportData('Indstrl_Mnth')

In [32]:
exportData('Mtrls_Mnth')

In [33]:
exportData('Tech_Mnth')

In [34]:
exportData('Util_Mnth')

In [46]:
# combined dataset

# input
CD = pd.read_csv("data/CD_Mnth_predictive_analytics_data.csv")
CS = pd.read_csv("data/CS_Mnth_predictive_analytics_data.csv")
energy = pd.read_csv("data/Enrgy_Mnth_predictive_analytics_data.csv")
financial = pd.read_csv("data/Fncl_Mnth_predictive_analytics_data.csv")
health = pd.read_csv("data/HlthC_Mnth_predictive_analytics_data.csv")
industrial = pd.read_csv("data/Indstrl_Mnth_predictive_analytics_data.csv")
materials = pd.read_csv("data/Mtrls_Mnth_predictive_analytics_data.csv")
tech = pd.read_csv("data/Tech_Mnth_predictive_analytics_data.csv")
utilities = pd.read_csv("data/Util_Mnth_predictive_analytics_data.csv")

# concatenate
combined_data = pd.concat([CD,CS,energy,financial,health,industrial,
                                materials,tech,utilities])

combined_data.head()
combined_data = combined_data.drop(columns=['Unnamed: 0'])
combined_data=combined_data.reset_index()
combined_data=combined_data.drop(columns=['index'])
combined_data

,Date,Actual,Predicted,Sector Code
0,2/1/2003,-0.0420,-0.0050,CD_Mnth
1,3/1/2003,0.0022,-0.0420,CD_Mnth
2,4/1/2003,0.0147,0.0022,CD_Mnth
3,5/1/2003,0.1170,-0.0034,CD_Mnth
4,6/1/2003,0.0654,0.0673,CD_Mnth
...,...,...,...,...
1912,6/1/2020,0.0544,0.0076,Util_Mnth
1913,7/1/2020,-0.0535,0.0191,Util_Mnth
1914,8/1/2020,0.0763,0.0116,Util_Mnth
1915,9/1/2020,-0.0285,0.0544,Util_Mnth


In [47]:
# output
combined_data.to_csv("data/COMBINED_DATA.csv", index=False, header=True)